In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/attaindb/dbo-employee_type_pay_type-210-live.1690705553.csv
/kaggle/input/attaindb/dbo-message_template-196-live.1690709451.csv
/kaggle/input/attaindb/dbo-employee_payroll_status-245-live.1690705214.csv
/kaggle/input/attaindb/dbo-candidate_job_application-175-live.1690703569.csv
/kaggle/input/attaindb/dbo-payments-181-live.1690885843.csv
/kaggle/input/attaindb/dbo-employee_job_assignment-98-live.1690704827.csv
/kaggle/input/attaindb/dbo-ep_review_type-268-live.1690705782.csv
/kaggle/input/attaindb/dbo-leave_request-110-live.1690709248.csv
/kaggle/input/attaindb/dbo-shift-205-live.1690886426.csv
/kaggle/input/attaindb/dbo-employee_change_log-247-live.1690704702.csv
/kaggle/input/attaindb/dbo-job_offer_candidate_assignment-179-live.1690708421.csv
/kaggle/input/attaindb/dbo-user_onboarding_task-184-live.1690887111.csv
/kaggle/input/attaindb/dbo-employee_review_actions-264-live.1690705333.csv
/kaggle/input/attaindb/dbo-employee_offboarding_plan-154-live.1690705138.csv
/kaggle

In [4]:
!pip install transformers
from transformers import TapasConfig, TapasForQuestionAnswering, AdamW



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
table=pd.read_csv("/kaggle/input/attaindb/dbo-user_roles-121-live.1690887165.csv")

In [6]:
table.head(20)

,id,created_at,updated_at,role_id,user_id,organization_id
0,1,1678188102403,NaN,2.0,2.0,1
1,2,1678188748408,NaN,2.0,1.0,1
2,3,1678364291689,NaN,4.0,3.0,1
3,4,1679577582879,NaN,4.0,2.0,1
4,5,1683981876930,NaN,6.0,2.0,1
5,6,1683982999742,NaN,1.0,1.0,1
6,7,1683989517132,NaN,8.0,2.0,1
7,11,1683996515849,NaN,7.0,2.0,1
8,13,1684002096221,NaN,5.0,2.0,1
9,14,1684002898164,NaN,8.0,1.0,1


In [7]:
from transformers import TapasTokenizer

# Load the TapasTokenizer
tokenizer = TapasTokenizer.from_pretrained("google/tapas-base")

In [8]:
# Generate a list of questions for each column and row combination
questions = []
for col in table.columns:
    for i, row in table.iterrows():
        questions.append(f"What is the value in the {col} column for row {i + 1}?")

# Generate corresponding answer coordinates and answer text
answer_coordinates = [[(i, col_idx)] for col_idx in range(len(table.columns)) for i in range(len(table))]
answer_text = [str(table.iat[i, col_idx]) for col_idx in range(len(table.columns)) for i in range(len(table))]


In [9]:
# Tokenize the data
inputs = tokenizer(
    table=table.astype(str),  # Convert table data to text-only
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)

In [10]:
import torch

class TableDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

    def __len__(self):
        return len(self.inputs["input_ids"])

# Create the DataLoader
train_dataset = TableDataset(inputs, tokenizer)
batch_size = 8  # You can adjust this based on your resources
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
from transformers import TapasConfig, TapasForQuestionAnswering, AdamW

# Define the configuration
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
    
)

# Initialize the model
model = TapasForQuestionAnswering.from_pretrained("google/tapas-base", config=config)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Put the model in training mode
model.train()

# Training loop
epochs = 3  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_dataloader:
        # Extract inputs from the batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        #float_answer = batch["float_answer"]
        batch_size = input_ids.shape[0]
        dummy_float_answers = torch.zeros(batch_size, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=dummy_float_answers,
            
        )

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()


Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at google/tapas-base and are newly initialized: ['aggregation_classifier.bias', 'column_output_weights', 'aggregation_classifier.weight', 'output_weights', 'column_output_bias', 'output_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
output_dir = "/kaggle/working/model"  # Modify this path accordingly
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/kaggle/working/model/tokenizer_config.json',
 '/kaggle/working/model/special_tokens_map.json',
 '/kaggle/working/model/vocab.txt',
 '/kaggle/working/model/added_tokens.json')

2. Finetune for Approval table 
/kaggle/input/attaindb/dbo-candidate-174-live.1690703493.csv

In [15]:
table=pd.read_csv("/kaggle/input/attaindb/dbo-candidate-174-live.1690703493.csv")

In [16]:
table.head()

,id,created_at,candidate_code,first_name,last_name,middle_name,full_name,initials,gender,birth_date,...,home_address,email,legal_id_type,legal_id_number,stage_id,created_by,updated_at,updated_by,organization_id,Source
0,1,1690395114858,233.0,Adepeu,Ajiba,Ola,Adepe Aji,A.O,Male,2000-01-02,...,"Iba, Oyo",adepe@gmail.com,NIN,54352525252,1,1,2.0,1,1,Linkedin
1,2,1685772573008,3333.0,Wale,Ade,NaN,Wale Ade,QA,Male,1985-04-16,...,Abeokuta,waleade@gmail.com,NaN,0,2,0,NaN,0,1,Job Page
2,3,1690510078908,2333.0,Adepeluuu,Ajiii,Ola,Adepeluuuu Ajiii,A.A,Male,2000-01-01,...,"Ibadan, Oyo",ade@gmail.com,NIN,5435352525252,1,1,2.0,1,1,Linkedin
3,4,1685773187634,5666.0,Blessing,Ojo,NaN,Blessing Ojo,BO,Female,1989-12-14,...,Benin City,blessing@gmail.com,NaN,0,3,0,NaN,0,1,Referrals
4,5,1685773298907,3444.0,Nancy,Ada,Ngozi,Nancy Ada,NaN,Female,1991-09-18,...,Enugu,ngoziada@gmail.com,NaN,0,2,0,NaN,0,1,Indeed


In [17]:
# Generate questions
questions = []
answer_coordinates = []
answer_text = []

for i, row in table.iterrows():
    questions.append(f"What is the full name for candidate {row['full_name']} (row {i + 1})?")
    answer_coordinates.append([(i, table.columns.get_loc("full_name"))])
    answer_text.append(str(table.iat[i, table.columns.get_loc("full_name")]))
    
    questions.append(f"What is the email for candidate {row['full_name']} (row {i + 1})?")
    answer_coordinates.append([(i, table.columns.get_loc("email"))])
    answer_text.append(str(table.iat[i, table.columns.get_loc("email")]))

In [18]:
# Tokenize the data
tokenizer = TapasTokenizer.from_pretrained("/kaggle/working/model")
inputs = tokenizer(
    table=table.astype(str),  # Convert table data to text-only
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding= True,
    return_tensors="pt",
)

Token indices sequence length is longer than the specified maximum sequence length for this model (873 > 512). Running this sequence through the model will result in indexing errors.


In [19]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

    def __len__(self):
        return len(self.inputs["input_ids"])

# Create the DataLoader
train_dataset = TableDataset(inputs, tokenizer)
batch_size = 8  # You can adjust this based on your resources
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



In [20]:
from transformers import TapasConfig, TapasForQuestionAnswering, AdamW

# Define the configuration
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
    
)

In [21]:
model = TapasForQuestionAnswering.from_pretrained("/kaggle/working/model", config=config)


In [25]:
# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Put the model in training mode
model.train()

# Training loop
epochs = 3  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_dataloader:
        # Extract inputs from the batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        #float_answer = batch["float_answer"]
        batch_size = input_ids.shape[0]
        dummy_float_answers = torch.zeros(batch_size, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=dummy_float_answers,
            
        )

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()



In [43]:
output_dir = "/kaggle/working/model"  # Modify this path accordingly
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/kaggle/working/model/tokenizer_config.json',
 '/kaggle/working/model/special_tokens_map.json',
 '/kaggle/working/model/vocab.txt',
 '/kaggle/working/model/added_tokens.json')

For Employee paytypes 

In [44]:
table= pd.read_csv("/kaggle/input/attaindb/dbo-employee_paytypes-218-live.1690705274.csv")

In [45]:
table.head()

,id,created_at,paytypes_id,employee_id,amount,status,start_date,end_date,updated_at,created_by,updated_by,organization_id,pay_schedule_id,payroll_config_id
0,1,1684317020263,4,87,500000,active,NaN,NaN,NaN,NaN,NaN,1,1,2
1,3,1684317931161,3,92,0,active,2023-05-17,2024-05-03,NaN,faruq,faruq,6,0,0
2,5,1685207905335,5,91,40000,active,2023-06-05,2023-06-05,1.685975e+12,Zulikif,Zulikif,1,0,0
3,6,1685975480538,3,93,7000,active,2023-05-17,2024-09-20,1.685976e+12,Zulikif,Zulikif,1,0,0


In [46]:
# Generate questions, answer coordinates, and answer text
questions = []
answer_coordinates = []
answer_text = []

# Questions about the status of an employee paytype
for i, row in table.iterrows():
    questions.append(f"What is the status of employee {row['employee_id']}'s paytype?")
    answer_coordinates.append([(i, table.columns.get_loc('status'))])
    answer_text.append(row['status'])

# Questions about the amount
for i, row in table.iterrows():
    questions.append(f"What is the amount for employee {row['employee_id']}?")
    answer_coordinates.append([(i, table.columns.get_loc('amount'))])
    answer_text.append(str(row['amount']))

In [47]:

inputs = tokenizer(
    table=table.astype(str),  # Convert table data to text-only
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding= "max_length",
    return_tensors="pt",
)

In [49]:
import torch

class TableDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

    def __len__(self):
        return len(self.inputs["input_ids"])

# Create the DataLoader
train_dataset = TableDataset(inputs, tokenizer)
batch_size = 8  # You can adjust this based on your resources
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [50]:
# Define the configuration
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
    
)

In [51]:
# Initialize the model
model = TapasForQuestionAnswering.from_pretrained("/kaggle/working/model", config=config)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [52]:
# Put the model in training mode
model.train()

# Training loop
epochs = 3  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_dataloader:
        # Extract inputs from the batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        #float_answer = batch["float_answer"]
        batch_size = input_ids.shape[0]
        dummy_float_answers = torch.zeros(batch_size, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=dummy_float_answers,
            
        )

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()


output_dir = "/kaggle/working/model"  # Modify this path accordingly
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/kaggle/working/model/tokenizer_config.json',
 '/kaggle/working/model/special_tokens_map.json',
 '/kaggle/working/model/vocab.txt',
 '/kaggle/working/model/added_tokens.json')

For employee Review Question 

In [53]:
table=pd.read_csv("/kaggle/input/attaindb/dbo-ep_review_question-270-live.1690705733.csv")

In [54]:
table.head()

,id,created_at,text,response_type,review_id,organization_id
0,12,1689435910841,How are you doing in the job?,rating,10,1
1,13,1689435948936,How are you doing in the job?,rating,11,1
2,14,1689435966827,How many courses did you take?,rating,11,1
3,15,1689740506227,How do you feel about your job,text,12,1


In [55]:
questions = []
answer_coordinates = []
answer_text = []

for i, row in table.iterrows():
    question_text = f"What is the 'text' for review ID {row['review_id']}?"
    question_response_type = f"What is the 'response_type' for review ID {row['review_id']}?"
    
    questions.extend([question_text, question_response_type])
    
    answer_coordinates.extend([[(i, table.columns.get_loc('text'))], [(i, table.columns.get_loc('response_type'))]])
    answer_text.extend([row['text'], row['response_type']])

In [56]:
# Tokenize the data
tokenizer = TapasTokenizer.from_pretrained("/kaggle/working/model")
inputs = tokenizer(
    table=table.astype(str),  # Convert table data to text-only
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)


In [57]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

    def __len__(self):
        return len(self.inputs["input_ids"])

# Create the DataLoader
train_dataset = TableDataset(inputs, tokenizer)
batch_size = 8  # You can adjust this based on your resources
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [58]:
# Define the configuration
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
    
)

In [60]:
model = TapasForQuestionAnswering.from_pretrained("/kaggle/working/model", config=config)
# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [61]:
# Put the model in training mode
model.train()

# Training loop
epochs = 3  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_dataloader:
        # Extract inputs from the batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        #float_answer = batch["float_answer"]
        batch_size = input_ids.shape[0]
        dummy_float_answers = torch.zeros(batch_size, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=dummy_float_answers,
            
        )

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()


output_dir = "/kaggle/working/model"  # Modify this path accordingly
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/kaggle/working/model/tokenizer_config.json',
 '/kaggle/working/model/special_tokens_map.json',
 '/kaggle/working/model/vocab.txt',
 '/kaggle/working/model/added_tokens.json')

Fou User Onboarding 

In [62]:
table=pd.read_csv("/kaggle/input/attaindb/dbo-user_onboarding_task-184-live.1690887111.csv")

In [63]:
table.head()

,id,created_at,task_name,description,sequence,organization_id
0,1,1685789586041,user_registration,NaN,0,1
1,2,1685789632035,org_registration,NaN,0,1
2,3,1685789651204,app_selection,NaN,0,1
3,4,1685789659039,NaN,NaN,0,0


In [64]:
questions = []
answer_coordinates = []
answer_text = []

for i, row in table.iterrows():
    if row['task_name'] is not None:
        question_task_name = f"What is the 'task_name' for ID {row['id']}?"
        questions.append(question_task_name)
        
        answer_coordinates.append([(i, table.columns.get_loc('task_name'))])
        answer_text.append(row['task_name'])

In [65]:
# Tokenize the data
tokenizer = TapasTokenizer.from_pretrained("/kaggle/working/model")
inputs = tokenizer(
    table=table.astype(str),  # Convert table data to text-only
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_text,
    padding="max_length",
    return_tensors="pt",
)


In [66]:
class TableDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, tokenizer):
        self.inputs = inputs
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

    def __len__(self):
        return len(self.inputs["input_ids"])

# Create the DataLoader
train_dataset = TableDataset(inputs, tokenizer)
batch_size = 8  # You can adjust this based on your resources
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [67]:
# Define the configuration
config = TapasConfig(
    num_aggregation_labels=4,
    use_answer_as_supervision=True,
    answer_loss_cutoff=0.664694,
    cell_selection_preference=0.207951,
    huber_loss_delta=0.121194,
    init_cell_selection_weights_to_zero=True,
    select_one_column=True,
    allow_empty_column_selection=False,
    temperature=0.0352513,
    
)

In [68]:
model = TapasForQuestionAnswering.from_pretrained("/kaggle/working/model", config=config)
# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [69]:
# Put the model in training mode
model.train()

# Training loop
epochs = 3  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_dataloader:
        # Extract inputs from the batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]
        labels = batch["labels"]
        numeric_values = batch["numeric_values"]
        numeric_values_scale = batch["numeric_values_scale"]
        #float_answer = batch["float_answer"]
        batch_size = input_ids.shape[0]
        dummy_float_answers = torch.zeros(batch_size, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels,
            numeric_values=numeric_values,
            numeric_values_scale=numeric_values_scale,
            float_answer=dummy_float_answers,
            
        )

        # Compute loss and perform backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()


output_dir = "/kaggle/working/model"  # Modify this path accordingly
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/kaggle/working/model/tokenizer_config.json',
 '/kaggle/working/model/special_tokens_map.json',
 '/kaggle/working/model/vocab.txt',
 '/kaggle/working/model/added_tokens.json')